<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">به نام خدا</div></center>
<img src="./logo.png" alt="class.vision" style="width: 200px;"/>
<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">Seq2Seq برای جمع  اعداد!</div></center></h1>

In [1]:
from random import seed
from random import randint
from numpy import array
from math import ceil
from math import log10
from math import sqrt
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

Using TensorFlow backend.


In [2]:
# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

In [3]:
x,y = random_sum_pairs(100,3,15)
print(len(x))
print(x[0])
print(y[0])

100
[12, 3, 4]
19


In [4]:
# convert data to strings
def to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [5]:
# integer encode strings
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [6]:
 # one hot encode
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [7]:
# generate an encoded dataset
def generate_data(n_samples, n_numbers, largest, alphabet):
    # generate pairs
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    # convert to strings
    X, y = to_string(X, y, n_numbers, largest)
    # integer encode
    X, y = integer_encode(X, y, alphabet)
    # one hot encode
    X, y = one_hot_encode(X, y, len(alphabet))
    # return as numpy arrays
    X, y = array(X), array(y)
    return X, y

In [8]:
# invert encoding
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [9]:
# define dataset
seed(1)
n_samples = 1000
n_numbers = 2
largest = 10
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
n_chars = len(alphabet)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))

In [10]:
X, y = generate_data(n_samples, n_numbers, largest, alphabet)

In [11]:
print("shape of X", X.shape)
print("shape of y", y.shape)
print("X[0]:")
print(X[0])
print("y[0]")
print(y[0])

print("invert X[0]", invert(X[0], alphabet) )
print("invert y[0]", invert(y[0], alphabet) )

shape of X (1000, 5, 12)
shape of y (1000, 2, 12)
X[0]:
[[0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]]
y[0]
[[0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]]
invert X[0]  3+10
invert y[0] 13


In [12]:
# define LSTM configuration
n_batch = 10
n_epoch = 30

In [13]:
# create LSTM
model = Sequential()
model.add(LSTM(100, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               45200     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 2, 100)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 50)             30200     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2, 12)             612       
Total params: 76,012
Trainable params: 76,012
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# train LSTM
for i in range(n_epoch):
    X, y = generate_data(n_samples, n_numbers, largest, alphabet)
    print(i)
    model.fit(X, y, epochs=1, batch_size=n_batch)

0
Epoch 1/1
1000/1000 [==============================] - 4s 4ms/step - loss: 1.9967 - accuracy: 0.3445
1
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 1.5194 - accuracy: 0.3680
2
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 1.3945 - accuracy: 0.4670
3
Epoch 1/1
1000/1000 [==============================] - 3s 3ms/step - loss: 1.3099 - accuracy: 0.5075
4
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 1.1971 - accuracy: 0.5595
5
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 1.0768 - accuracy: 0.6285
6
Epoch 1/1
1000/1000 [==============================] - 3s 3ms/step - loss: 0.9106 - accuracy: 0.6870
7
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.7884 - accuracy: 0.7330
8
Epoch 1/1
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6850 - accuracy: 0.8075
9
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - lo

In [15]:
# evaluate on some new patterns
X, y = generate_data(n_samples, n_numbers, largest, alphabet)
result = model.predict(X, batch_size=n_batch, verbose=0)
# calculate error
expected = [invert(x, alphabet) for x in y]
predicted = [invert(x, alphabet) for x in result]
# show some examples
for i in range(20):
    print('Expected=%s, Predicted=%s' % (expected[i], predicted[i]))

Expected=14, Predicted=14
Expected=15, Predicted=15
Expected=12, Predicted=12
Expected=12, Predicted=12
Expected=13, Predicted=13
Expected=15, Predicted=15
Expected= 8, Predicted= 8
Expected= 4, Predicted= 4
Expected=19, Predicted=19
Expected=13, Predicted=13
Expected=15, Predicted=15
Expected= 8, Predicted= 8
Expected=13, Predicted=13
Expected= 4, Predicted= 4
Expected=16, Predicted=16
Expected=13, Predicted=13
Expected=16, Predicted=16
Expected= 8, Predicted= 8
Expected=14, Predicted=14
Expected=16, Predicted=16


### Extensions

This section lists some natural extensions to this tutorial that you may wish to explore.

   ***Integer Encoding***. Explore whether the problem can learn the problem better using an integer encoding alone. The ordinal relationship between most of the inputs may prove very useful.
    
   ***Variable Numbers***. Change the example to support a variable number of terms on each input sequence. This should be straightforward as long as you perform sufficient padding.
    
   ***Variable Mathematical Operations***. Change the example to vary the mathematical operation to allow the network to generalize even further.
    Brackets. Allow the use of brackets along with other mathematical operations.

Source:
    https://machinelearningmastery.com/learn-add-numbers-seq2seq-recurrent-neural-networks/

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma"> دوره پیشرفته یادگیری عمیق<br>علیرضا اخوان پور<br>  آبان و آذر 1399<br>
</div>
<a href="http://class.vision">Class.Vision</a> - <a href="http://AkhavanPour.ir">AkhavanPour.ir</a> - <a href="https://github.com/Alireza-Akhavan/">GitHub</a>

</div>